In [51]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

import time
import datetime

import json
import os
from dotenv import load_dotenv #pip install python-dotenv

from bs4 import BeautifulSoup
import requests 

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

from my_functions import *

### **``master_id = 0``**

In [15]:
url = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

# Define parameters for artist search
params = {
    "per_page": 100    # Number of results per page (max 100)
    , "page": 1 
    , 'type': 'release'
    , 'format': 'album'
    , 'sort': 'title'
    , 'sort_order': 'asc'
    , 'master_id': 0

    # items to change
    , 'genre': 'Rock'
    , 'country': "UK"
    , 'year': 2022
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

data['pagination']['items']

1191

In [7]:
14*3600/5

10080.0

**SUPER CODI**

In [22]:
url_search = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')

# rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
#               , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

# create empty lists
artist_id_list = []
artists_list = []
titles_list = []
years_list = []
release_countries_list = []
album_length_list = []
tracks_list = []
release_type_list = []
genres_list = []
styles_list = []
master_id_new_list = []
artist_profile_list = []
main_release_id_list = []

attempts = 0
scrapped = 0
start_year = 2004

for year in range(start_year, start_year+1):
    print(f'Year: {year}')

    # Define parameters for album search
    params = {
        "type": "release"  # Search for masters
        , "format": "album"           # just search for albums
        , "per_page": 100    # Number of results per page (max 100)
        , "page": 1          # Start with page 1
        , 'country': 'UK'
        , 'master_id': 0
        , "year": year
        , 'genre': 'Rock'
    }

    response1 = requests.get(url_search, headers=headers, params=params)
    time.sleep(1)
    data1 = response1.json()

    items = data1['pagination']['items']
    print(f'Items: {items} releases')

    pages = data1['pagination']['pages']+1
    
    for page in range(1, pages):
        print(f'------------ Page: {page} of {pages-1}, {year} -------------')

        params['page'] = page     # change pages
        response2 = requests.get(url_search, headers=headers, params=params)
        data2 = response2.json()

        for id in range(len(data2['results'])):
            try:
                attempts+=1
                time.sleep(1)

                release_id = data2['results'][id]['id']

                url_release = f"https://api.discogs.com/releases/{release_id}"
                response3 = requests.get(url_release, headers=headers)
                time.sleep(1)
                data3 = response3.json()
                artist = data3['artists'][0]['name']

                if artist == 'Various':
                        print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - release_id {release_id}: Compilation') 
                        continue   # I don't want compilations
                else:
                # define the data I want to get
                    title = data3['title']
                    release_year = data3['year']                
                    genres = data3['genres']
                    styles = data3['styles']

                # a more complex information is the total duration of the album, I have to sum the durations of each song
                    number_of_songs = len(data3['tracklist'])
                    song_durations = []

                    for song in range(number_of_songs):
                        song_duration = data3['tracklist'][song]['duration']
                        try:
                            minutes = int(song_duration.split(':')[0])
                            seconds = int(song_duration.split(':')[1])
                            song_duration_minutes = minutes + seconds/60
                            song_durations.append(song_duration_minutes)
                        except:
                            pass
                        album_length = round(sum(i for i in song_durations), 2)

                    artist_id = data3['artists'][0]['id']

                # define the data I want to get
                    release_country = data3['country']
                    release_type = data3['formats'][0]['descriptions']

                # artist page
                    url = f"https://api.discogs.com/artists/{artist_id}"
                    time.sleep(1)
                    response5 = requests.get(url, headers=headers)
                    data5 = response5.json()

                # define the data I want to get
                    try:
                        artist_profile = data5['profile']
                    except:
                        artist_profile = np.nan
                        pass

                # save the info in the lists
                    artist_id_list.append(str(artist_id))
                    artists_list.append(artist)
                    titles_list.append(title)
                    years_list.append(release_year)
                    genres_list.append(genres)
                    styles_list.append(styles)
                    master_id_new_list.append(np.nan)
                    album_length_list.append(album_length)
                    tracks_list.append(number_of_songs)
                    release_countries_list.append(release_country)
                    release_type_list.append(release_type)
                    main_release_id_list.append(release_id)

                    if artist_profile == '':
                        artist_profile_list.append(np.nan)
                    else:
                        artist_profile_list.append(artist_profile)

                    scrapped+=1

                # list lengths
                    lists = [artist_id_list, main_release_id_list, master_id_new_list, release_countries_list, artists_list, titles_list, album_length_list
                            , tracks_list, release_type_list, genres_list, styles_list, artist_profile_list, years_list]

                # Check if all lists have the same length
                    lengths = [len(lst) for lst in lists]
                    if len(set(lengths)) != 1:
                        print("Lengths are not the same.")
                        break # stop the loop, I won't be able to store the data if I have one value missing

                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - release_id {release_id}: {artist} - {title}')
                    time.sleep(1)
            except:
                print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - release_id {release_id}: {artist} - {title} - Album not found')
                continue    # sometimes it crashes for some unknown reason, so I skip this album and continue

Year: 2004
Items: 1058 releases
------------ Page: 1 of 11, 2004 -------------
1 attempts - 1 scrapped - 1/1058 - release_id 4623549: Robin George - Dangerous Music Live '85
2 attempts - 2 scrapped - 2/1058 - release_id 676122: Voices Of Masada - Four Corners
3 attempts - 3 scrapped - 3/1058 - release_id 2040777: Beef Conspiracy - Hung, Drawn And Quarter Poundered
4 attempts - 4 scrapped - 4/1058 - release_id 3420766: Mendeed - From Shadows Came Darkness
5 attempts - 5 scrapped - 5/1058 - release_id 3222524: Adrian Shaw - String Theory
6 attempts - 6 scrapped - 6/1058 - release_id 4425230: Tyla - Fakin' Kant
7 attempts - 7 scrapped - 7/1058 - release_id 4005068: Red Flag 77 - Stop The World
8 attempts - 8 scrapped - 8/1058 - release_id 4528442: Deliverance (15) - The Executioner
9 attempts - 9 scrapped - 9/1058 - release_id 8717899: The Infernos (4) - Don't Know What You're Missin'
10 attempts - 10 scrapped - 10/1058 - release_id 12135453: Stories And Comets - Time To Write You Out
11 

In [17]:
(65*60)/851

4.582843713278496

#### Continue

In [27]:
df_masters_id0 = pd.DataFrame({'artist_id': artist_id_list
                        , 'master_id': master_id_new_list
                        , 'main_release_id': main_release_id_list
                        , 'release_country': release_countries_list
                        , 'artist': artists_list
                        , 'title': titles_list
                        , 'year': years_list
                        , 'album_length': album_length_list
                        , 'tracks': tracks_list
                        , 'release_type': release_type_list
                        , 'genres': genres_list
                        , 'styles': styles_list
                        , 'artist_profile': artist_profile_list
                        })
df_masters_id0.shape

(971, 13)

In [28]:
print(f'{df_masters_id0.shape[0]} releases\n{df_masters_id0.artist.nunique()} different artists')

971 releases
757 different artists


In [29]:
df_masters_id0

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
0,585672,NaN,4623549,UK,Robin George,Dangerous Music Live '85,2004,48.60,12,[Album],[Rock],[Pop Rock],"British rock guitarist, singer, songwriter, an..."
1,447124,NaN,676122,UK,Voices Of Masada,Four Corners,2004,42.25,8,[Album],[Rock],[Goth Rock],"""Voices of Masada stop the clock of gothic mus..."
2,673899,NaN,2040777,UK,Beef Conspiracy,"Hung, Drawn And Quarter Poundered",2004,26.42,9,[Album],[Rock],[Death Metal],Death metal band from the United Kingdom.
3,311084,NaN,3420766,UK,Mendeed,From Shadows Came Darkness,2004,36.18,7,[Album],[Rock],[Heavy Metal],Metalcore band from Glasgow (Scotland) formed ...
4,329546,NaN,3222524,UK,Adrian Shaw,String Theory,2004,64.12,10,[Album],[Rock],"[Space Rock, Psychedelic Rock, Acid Rock]","Born 1947, in Hampstead, London, Adrian Shaw h..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,266538,NaN,4844952,UK & Europe,The Incredible String Band,The Incredible String Band & The 5000 Spirits ...,2004,0.00,31,"[Compilation, Album]","[Rock, Folk, World, & Country]","[Folk, Folk Rock, Acoustic, Psychedelic Rock]",Scottish psychedelic folk band from Edinburgh ...
967,406171,NaN,2481542,UK & Europe,Tom Paxton,Outward Bound & Morning Again,2004,99.80,39,"[Album, Compilation, Remastered]","[Rock, Folk, World, & Country]","[Folk Rock, Acoustic, Folk]","US folksinger and songwriter, born October 31,..."
968,1120586,NaN,4659274,UK & Europe,David Daltrey,The Wayfarer,2004,0.00,39,[Album],"[Jazz, Rock, Funk / Soul, Blues, Pop, Folk, Wo...",[],NaN
969,6054632,NaN,20223706,UK & Ireland,Captain Kidder,Right On Under,2004,43.07,16,"[Album, Compilation, Limited Edition, Stereo]","[Rock, Pop, Folk, World, & Country]","[Acoustic, Alternative Rock, Lo-Fi, Avantgarde]",NaN


### **Exporting to csv**

In [30]:
df_masters_id0_scrapped = pd.read_csv('Datasets/df_masters_id0.csv')
df_masters_id0_scrapped.shape

(53, 16)

In [43]:
df_masters_id0_concat = pd.concat([df_masters_id0_scrapped, df_masters_id0])
df_masters_id0_concat.reset_index(drop=True, inplace=True)
df_masters_id0_concat.shape

(1024, 13)

In [54]:
df_masters_id0_concat.year.value_counts()

year
2004    791
2024     21
2002     12
2000      9
2003      7
2001      4
Name: count, dtype: int64

In [52]:
check_duplicates_albums(df_masters_id0_concat)

Found 180 duplicates
Resulting dataset: (844, 13)


In [53]:
df_masters_id0_concat.to_csv('Datasets/df_masters_id0.csv', index=False)